# Vector Memory

The vector memory module uses vector search (backed by a vector db) to retrieve relevant conversation items given a user input.

This notebook shows you how to use the `VectorMemory` class. We show you how to use its individual functions, as well as how to plug it into an agent.

### Initialize and Experiment with Memory Module

Here we initialize a raw memory module and demonstrate its functions - to put and retrieve from ChatMessage objects.

- Note that `retriever_kwargs` is the same args you'd specify on the `VectorIndexRetriever` or from `index.as_retriever(..)`.

In [1]:
from llama_index.core.memory import VectorMemory
from llama_index.embeddings.openai import OpenAIEmbedding


vector_memory = VectorMemory.from_defaults(
    vector_store=None, # leave as None to use default in-memory vector store
    embed_model=OpenAIEmbedding(),
    retriever_kwargs={"similarity_top_k": 1} 
)

In [2]:
from llama_index.core.llms import ChatMessage

msgs = [
    ChatMessage.from_str("Jerry likes juice.", "user"),
    ChatMessage.from_str("Bob likes burgers.", "user"),
    ChatMessage.from_str("Alice likes apples.", "user")
]

In [3]:
# load into memory
for m in msgs:
    vector_memory.put(m)

In [4]:
# retrieve from memory
msgs = vector_memory.get("What does Jerry like?")
msgs

Memory returning messages: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='This is a set of relevant messages retrieved from longer-term history: ', additional_kwargs={}), ChatMessage(role=<MessageRole.USER: 'user'>, content='Jerry likes juice.', additional_kwargs={})]


[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='This is a set of relevant messages retrieved from longer-term history: ', additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content='Jerry likes juice.', additional_kwargs={})]

In [5]:
vector_memory.reset()

Note that the system message is by default returned along with the user message. You can choose to disable the system message if you'd like.

Now let's try resetting and trying again. This time, we'll add an assistant message. Note that user/assistant messages are bundled by default.

In [6]:
msgs = [
    ChatMessage.from_str("Jerry likes burgers.", "user"),
    ChatMessage.from_str("Bob likes apples.", "user"),
    ChatMessage.from_str("Indeed, Bob likes apples.", "assistant"),
    ChatMessage.from_str("Alice likes juice.", "user")
]
vector_memory.set(msgs)
vector_memory.system_message = False

In [7]:
msgs = vector_memory.get("What does Bob like?")
msgs

Memory returning messages: [ChatMessage(role=<MessageRole.USER: 'user'>, content='Bob likes apples.', additional_kwargs={}), ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='Indeed, Bob likes apples.', additional_kwargs={})]


[ChatMessage(role=<MessageRole.USER: 'user'>, content='Bob likes apples.', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='Indeed, Bob likes apples.', additional_kwargs={})]

## Plug Memory Module into Agent

Let's now try plugging this memory module into an agent. 

In [8]:
vector_memory.reset()

In [9]:
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import FunctionCallingAgentWorker

import nest_asyncio

nest_asyncio.apply()

In [10]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b

def mystery(a: int, b: int) -> int:
    """Mystery function on two numbers"""
    return a**2 - b**2


multiply_tool = FunctionTool.from_defaults(fn=multiply)
mystery_tool = FunctionTool.from_defaults(fn=mystery)

In [11]:
llm = OpenAI(model="gpt-3.5-turbo-0613")
agent_worker = FunctionCallingAgentWorker.from_tools(
    [multiply_tool, mystery_tool], llm=llm, verbose=True
)
agent = agent_worker.as_agent(memory=vector_memory)

In [12]:
response = agent.chat("What is the mystery function on 5 and 6?")
print(str(response))

Added user message to memory: What is the mystery function on 5 and 6?
Memory returning messages: []
=== Calling Function ===
Calling function: mystery with args: {"a": 5, "b": 6}
=== Function Output ===
-11
Memory returning messages: []
=== LLM Response ===
The mystery function on 5 and 6 returns -11.
assistant: The mystery function on 5 and 6 returns -11.


In [13]:
agent.memory.get("mystery")

Memory returning messages: [ChatMessage(role=<MessageRole.USER: 'user'>, content='What is the mystery function on 5 and 6?', additional_kwargs={}), ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=None, additional_kwargs={'tool_calls': [{'id': 'call_ZivV7lGHvE4qHW7H7DUKFSuy', 'function': {'arguments': '{\n  "a": 5,\n  "b": 6\n}', 'name': 'mystery'}, 'type': 'function'}]}), ChatMessage(role=<MessageRole.TOOL: 'tool'>, content='-11', additional_kwargs={'name': 'mystery', 'tool_call_id': 'call_ZivV7lGHvE4qHW7H7DUKFSuy'}), ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='The mystery function on 5 and 6 returns -11.', additional_kwargs={})]


[ChatMessage(role=<MessageRole.USER: 'user'>, content='What is the mystery function on 5 and 6?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=None, additional_kwargs={'tool_calls': [{'id': 'call_ZivV7lGHvE4qHW7H7DUKFSuy', 'function': {'arguments': '{\n  "a": 5,\n  "b": 6\n}', 'name': 'mystery'}, 'type': 'function'}]}),
 ChatMessage(role=<MessageRole.TOOL: 'tool'>, content='-11', additional_kwargs={'name': 'mystery', 'tool_call_id': 'call_ZivV7lGHvE4qHW7H7DUKFSuy'}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='The mystery function on 5 and 6 returns -11.', additional_kwargs={})]

In [14]:
response = agent.chat("What happens if you multiply 2 and 3?")

Added user message to memory: What happens if you multiply 2 and 3?
Memory returning messages: [ChatMessage(role=<MessageRole.USER: 'user'>, content='What is the mystery function on 5 and 6?', additional_kwargs={}), ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=None, additional_kwargs={'tool_calls': [{'id': 'call_ZivV7lGHvE4qHW7H7DUKFSuy', 'function': {'arguments': '{\n  "a": 5,\n  "b": 6\n}', 'name': 'mystery'}, 'type': 'function'}]}), ChatMessage(role=<MessageRole.TOOL: 'tool'>, content='-11', additional_kwargs={'name': 'mystery', 'tool_call_id': 'call_ZivV7lGHvE4qHW7H7DUKFSuy'}), ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='The mystery function on 5 and 6 returns -11.', additional_kwargs={})]
=== Calling Function ===
Calling function: multiply with args: {"a": 2, "b": 3}
=== Function Output ===
6
Memory returning messages: [ChatMessage(role=<MessageRole.USER: 'user'>, content='What is the mystery function on 5 and 6?', additional_kwargs={}), ChatM

In [15]:
agent.memory.get("mystery")

Memory returning messages: [ChatMessage(role=<MessageRole.USER: 'user'>, content='What is the mystery function on 5 and 6?', additional_kwargs={}), ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=None, additional_kwargs={'tool_calls': [{'id': 'call_ZivV7lGHvE4qHW7H7DUKFSuy', 'function': {'arguments': '{\n  "a": 5,\n  "b": 6\n}', 'name': 'mystery'}, 'type': 'function'}]}), ChatMessage(role=<MessageRole.TOOL: 'tool'>, content='-11', additional_kwargs={'name': 'mystery', 'tool_call_id': 'call_ZivV7lGHvE4qHW7H7DUKFSuy'}), ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='The mystery function on 5 and 6 returns -11.', additional_kwargs={})]


[ChatMessage(role=<MessageRole.USER: 'user'>, content='What is the mystery function on 5 and 6?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=None, additional_kwargs={'tool_calls': [{'id': 'call_ZivV7lGHvE4qHW7H7DUKFSuy', 'function': {'arguments': '{\n  "a": 5,\n  "b": 6\n}', 'name': 'mystery'}, 'type': 'function'}]}),
 ChatMessage(role=<MessageRole.TOOL: 'tool'>, content='-11', additional_kwargs={'name': 'mystery', 'tool_call_id': 'call_ZivV7lGHvE4qHW7H7DUKFSuy'}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='The mystery function on 5 and 6 returns -11.', additional_kwargs={})]

In [16]:
print(str(response))

assistant: If you multiply 2 and 3, the result is 6.


In [17]:
response = agent.chat("?")

Added user message to memory: What if you multiply the output of the mystery operation from the previous message, by 5?
Memory returning messages: [ChatMessage(role=<MessageRole.USER: 'user'>, content='What happens if you multiply 2 and 3?', additional_kwargs={}), ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=None, additional_kwargs={'tool_calls': [{'id': 'call_8LH43Pzh6hW2kJQKkCiAwmyX', 'function': {'arguments': '{\n  "a": 2,\n  "b": 3\n}', 'name': 'multiply'}, 'type': 'function'}]}), ChatMessage(role=<MessageRole.TOOL: 'tool'>, content='6', additional_kwargs={'name': 'multiply', 'tool_call_id': 'call_8LH43Pzh6hW2kJQKkCiAwmyX'}), ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='If you multiply 2 and 3, the result is 6.', additional_kwargs={})]
=== Calling Function ===
Calling function: mystery with args: {"a": 2, "b": 3}
=== Function Output ===
-5
Memory returning messages: [ChatMessage(role=<MessageRole.USER: 'user'>, content='What happens if you multi